Перед запуском убедитесь, что в корне проекта есть файл .env и в нем заполнены выданные вам креды подключения к базам данных и хранилищу

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine

In [3]:
# подгружаем .env
load_dotenv()

True

In [4]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

mle_telegram_token = os.environ.get('MLE_TELEGRAM_TOKEN')
mle_telegram_chat_id = os.environ.get('MLE_TELEGRAM_CHAT_ID')

In [5]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [ ]:
# Пример выгрузки данных из БД
TABLE = ''
SQL = f'select * from {TABLE}'
data = pd.read_sql(SQL, src_conn)

In [6]:
import pandas as pd
from sqlalchemy import create_engine

# Создаем соединение
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

# 2. Получаем список таблиц
tables_query = """
SELECT tablename 
FROM pg_catalog.pg_tables 
WHERE schemaname = 'public' 
  AND tableowner != 'postgres'
ORDER BY tablename;
"""

tables = pd.read_sql(tables_query, dst_conn)
print("Список таблиц в базе:")
display(tables)

# 3. Для каждой таблицы показываем структуру и данные
for table in tables['tablename']:
    print(f"\n{'-'*50}")
    print(f"Таблица: {table}")
    
    # 3.1 Получаем структуру таблицы
    try:
        columns_query = f"""
        SELECT 
            column_name AS "Столбец",
            data_type AS "Тип",
            is_nullable AS "Null?"
        FROM information_schema.columns
        WHERE table_name = '{table}';
        """
        
        columns = pd.read_sql(columns_query, dst_conn)
        print("\nСтруктура:")
        display(columns)
    
    except Exception as e:
        print(f"Ошибка получения структуры: {str(e)}")
        continue
    
    # 3.2 Получаем первые 10 строк
    try:
        data_query = f"SELECT * FROM {table} LIMIT 10;"
        data = pd.read_sql(data_query, dst_conn)
        print(f"\nПервые 10 строк ({len(data)} всего):")
        display(data)
    
    except Exception as e:
        print(f"Ошибка получения данных: {str(e)}")

Список таблиц в базе:


,tablename
0,alt_users_churn
1,buildings
2,clean_users_churn
3,flats
4,merged_flats_buildings
5,users_churn



--------------------------------------------------
Таблица: alt_users_churn

Структура:


,Столбец,Тип,Null?
0,id,integer,NO
1,begin_date,timestamp without time zone,YES
2,end_date,timestamp without time zone,YES
3,monthly_charges,double precision,YES
4,total_charges,double precision,YES
5,senior_citizen,integer,YES
6,target,integer,YES
7,partner,character varying,YES
8,dependents,character varying,YES
9,internet_service,character varying,YES



Первые 10 строк (10 всего):


...

,id,floor,is_apartment,kitchen_area,living_area,rooms,studio,total_area,price,building_id,build_year,building_type,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,0,9,False,9.90,19.900000,1,False,35.099998,9500000,6220,1965,6,55.717113,37.781120,2.64,84,12,True
1,1,7,False,0.00,16.600000,1,False,43.000000,13500000,18012,2001,2,55.794849,37.608013,3.00,97,10,True
2,2,9,False,9.00,32.000000,2,False,56.000000,13500000,17821,2000,4,55.740040,37.761742,2.70,80,10,True
3,3,1,False,10.10,43.099998,3,False,76.000000,20000000,18579,2002,4,55.672016,37.570877,2.64,771,17,True
4,4,3,False,3.00,14.000000,1,False,24.000000,5200000,9293,1971,1,55.808807,37.707306,2.60,208,9,True
5,5,9,False,0.00,0.000000,2,False,51.009998,8490104,23964,2017,4,55.724728,37.743069,2.70,192,17,True
6,6,1,False,6.18,29.340000,2,False,44.520000,9500000,5576,1964,4,55.795589,37.722622,2.64,180,5,False
7,7,7,False,13.50,0.000000,1,False,52.000000,17990000,23007,2015,2,55.656345,37.424335,3.00,512,11,True
8,8,7,False,8.18,19.100000,1,False,35.919998,6300000,13491,1982,4,55.574734,37.668686,2.64,127,16,True
9,9,5,False,8.00,30.000000,2,False,50.000000,5900000,13731,1982,4,55.994698,37.196686,2.64,142,12,True



--------------------------------------------------
Таблица: users_churn

Структура:


,Столбец,Тип,Null?
0,id,integer,NO
1,begin_date,timestamp without time zone,YES
2,end_date,timestamp without time zone,YES
3,monthly_charges,double precision,YES
4,total_charges,double precision,YES
5,senior_citizen,integer,YES
6,target,integer,YES
7,partner,character varying,YES
8,dependents,character varying,YES
9,internet_service,character varying,YES



Первые 10 строк (10 всего):


,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,1,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,...,No,No,No,No,Female,0,Yes,No,None,0
1,2,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,...,Yes,No,No,No,Male,0,No,No,No,0
2,3,3668-QPYBK,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,...,No,No,No,No,Male,0,No,No,No,1
3,4,7795-CFOCW,2016-05-01,NaT,One year,No,Bank transfer (automatic),42.30,1840.75,DSL,...,Yes,Yes,No,No,Male,0,No,No,None,0
4,5,9237-HQITU,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,Fiber optic,...,No,No,No,No,Female,0,No,No,No,1
5,6,9305-CDSKC,2019-03-01,2019-11-01,Month-to-month,Yes,Electronic check,99.65,820.50,Fiber optic,...,Yes,No,Yes,Yes,Female,0,No,No,Yes,1
6,7,1452-KIOVK,2018-04-01,NaT,Month-to-month,Yes,Credit card (automatic),89.10,1949.40,Fiber optic,...,No,No,Yes,No,Male,0,No,Yes,Yes,0
7,8,6713-OKOMC,2019-04-01,NaT,Month-to-month,No,Mailed check,29.75,301.90,DSL,...,No,No,No,No,Female,0,No,No,None,0
8,9,7892-POOKP,2017-07-01,2019-11-01,Month-to-month,Yes,Electronic check,104.80,3046.05,Fiber optic,...,Yes,Yes,Yes,Yes,Female,0,Yes,No,Yes,1
9,10,6388-TABGU,2014-12-01,NaT,One year,No,Bank transfer (automatic),56.15,3487.95,DSL,...,No,No,No,No,Male,0,No,Yes,No,0
